# LLM-Based Exercise Recommendation System
## Hybrid Architecture: Rule-Based + Two-LLM for Physiotherapy Exercise Prescription

**System Flow:**
1. Fetch patient data from Supabase (questionnaire, STS assessment, demographics)
2. **Rule-Based Analysis**: Identify biomechanical targets using clinical criteria
3. **LLM #1 (Exercise Recommendation Agent)**: Select 4 exercises addressing identified targets
4. **LLM #2 (Safety Verification Agent)**: Review and approve/modify/reject for safety
5. Display final prescription with clinical rationale

**Benefits of Hybrid Approach:**
- ✅ **Consistency**: Rule-based biomechanical target identification ensures reproducibility
- ✅ **Token Efficiency**: Reduces LLM token usage by ~100-200 tokens per request
- ✅ **Transparency**: Biomechanical targets explicitly shown before LLM reasoning
- ✅ **Clinical Reliability**: Combines deterministic rules with LLM clinical reasoning

---

## Cell 1: Setup and Initialization

In [1]:
import os
from dotenv import load_dotenv
from langchain_deepseek import ChatDeepSeek
from supabase import create_client
import json

# Load environment variables
load_dotenv()

# Get API keys
deepseek_api_key = os.environ.get("DEEPSEEK_API_KEY")
supabase_url = os.environ.get("VITE_SUPABASE_URL")
supabase_key = os.environ.get("VITE_SUPABASE_ANON_KEY")

# Initialize LLM
llm = ChatDeepSeek(
    model="deepseek-chat",
    api_key=deepseek_api_key,
    temperature=0  # Deterministic for clinical consistency
)

# Initialize Supabase client
supabase = create_client(supabase_url, supabase_key)

print("✓ LLM initialized (DeepSeek Chat)")
print("✓ Supabase client initialized")
print("\nReady to process patient data!")

✓ LLM initialized (DeepSeek Chat)
✓ Supabase client initialized

Ready to process patient data!


## Cell 2: Import Custom Modules

In [2]:
# Import data fetching module
from data_fetcher import (
    fetch_patient_data,
    fetch_all_exercises,
    structure_patient_profile,
    print_patient_summary
)

# Import LLM agents
from llm1_recommendation import (
    generate_exercise_recommendations,
    print_llm1_output
)
from llm2_safety_verification import (
    verify_safety_and_finalize,
    print_llm2_output
)

print("✓ All modules imported successfully")

✓ All modules imported successfully


## Cell 3: User Input - Select Patient

In [3]:
# Enter username from database
# Test team should use this after filling out the HTML questionnaire
username = input("Enter patient username from database: ")

print(f"\n📋 Fetching data for patient: {username}")
print("="*80)

Enter patient username from database: Test_05

📋 Fetching data for patient: Test_05


## Cell 4: Fetch and Structure Patient Data

In [4]:
try:
    # Fetch raw data from Supabase
    print("Fetching patient data from Supabase...")
    raw_data = fetch_patient_data(supabase, username)
    
    # Fetch exercise database
    print("Fetching exercise database...")
    exercises = fetch_all_exercises(supabase)
    print(f"Loaded {len(exercises)} exercises")
    
    # Structure data for LLM consumption
    print("\nStructuring patient profile...")
    patient_profile = structure_patient_profile(raw_data, exercises)
    
    # Print summary
    print_patient_summary(patient_profile)
    
    print("\n✓ Data fetching complete!")
    
except ValueError as e:
    print(f"❌ ERROR: {e}")
    print("\nPlease ensure:")
    print("1. The username exists in the database")
    print("2. Patient has completed questionnaire")
    print("3. Patient has completed STS assessment")
    print("4. Patient demographics are recorded")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

Fetching patient data from Supabase...
Fetching exercise database...
Loaded 32 exercises

Structuring patient profile...
PATIENT PROFILE SUMMARY

Demographics:
  Age: 82 | Gender: female
  Height: 153.0 cm | Weight: 48.0 kg

STS Assessment:
  Repetitions: 7
  HK Norm (Average): 11 - 14
  Performance: Below Average
  Knee alignment: valgus
  Trunk sway: present | Hip sway: present

Questionnaire Sections (normalized 0-100, higher=better):
  symptoms            :  40.0 (avg: 2.80)
  stiffness           :  50.0 (avg: 2.50)
  pain                :  88.9 (avg: 1.33)
  function_ADL        :  39.2 (avg: 2.82)
  function_sports     :  20.0 (avg: 3.40)
  quality_of_life     :  33.3 (avg: 3.00)

Position-Relevant Questions (0=None, 1=Mild, 2=Moderate, 3=Severe, 4=Extreme):
  Weight-bearing:
    F1: Descending stairs = 4
    F2: Ascending stairs = 4
    F4: Standing = 2
    SP1: Squatting = 3
    SP4: Twisting/pivoting on your injured knee = 3
  Quadruped:
    SP5: Kneeling = 3
  Lying: Safe by d

In [5]:
patient_profile

{'demographics': {'age': 82,
  'gender': 'female',
  'height_cm': 153.0,
  'weight_kg': 48.0,
  'date_of_birth': '1943-02-22'},
 'questionnaire_sections': {'symptoms': {'questions': ['s1',
    's2',
    's3',
    's4',
    's5'],
   'scores': [2, 3, 3, 3, 3],
   'avg': 2.8,
   'total': 14,
   'normalized_0_100': 40.0},
  'stiffness': {'questions': ['st1', 'st2'],
   'scores': [3, 2],
   'avg': 2.5,
   'total': 5,
   'normalized_0_100': 50.0},
  'pain': {'questions': ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9'],
   'scores': [3, 1, 1, 1, 1, 2, 1, 1, 1],
   'avg': 1.33,
   'total': 12,
   'normalized_0_100': 88.9},
  'function_ADL': {'questions': ['f1',
    'f2',
    'f3',
    'f4',
    'f5',
    'f6',
    'f7',
    'f8',
    'f9',
    'f10',
    'f11',
    'f12',
    'f13',
    'f14',
    'f15',
    'f16',
    'f17'],
   'scores': [4, 4, 3, 2, 4, 2, 2, 3, 4, 2, 4, 1, 2, 0, 3, 4, 4],
   'avg': 2.82,
   'total': 48,
   'normalized_0_100': 39.2},
  'function_sports': {'questions'

## Cell 4.5: Biomechanical Target Identification (Rule-Based)

This step uses rule-based logic to identify biomechanical targets before LLM analysis.

In [6]:
from biomechanical_analyzer import identify_biomechanical_targets, print_biomechanical_targets

print("\n" + "="*80)
print("STEP 1: Rule-Based Biomechanical Target Identification")
print("="*80 + "\n")

# Identify biomechanical targets using rule-based logic
biomechanical_targets = identify_biomechanical_targets(patient_profile)

# Print in human-readable format
print_biomechanical_targets(biomechanical_targets)

print("\n✓ Biomechanical targets identified!")
print("  These targets will be provided to LLM #1 for exercise selection.\n")


STEP 1: Rule-Based Biomechanical Target Identification

BIOMECHANICAL TARGET ANALYSIS (RULE-BASED)

3 biomechanical target(s) identified:

1. Issue: Dynamic knee instability (Valgus alignment - knock-knees)
   Strategy: Dynamic knee instability usually associates with weak core anti-rotation control, to prioritize exercises with `core_contra=true`. Prioritize exercises with high glute_med_min in muscles.primary_movers or muscles.secondary_movers (value 4-5). Match muscle role to functional capacity: for lower function patient, try to find those muscle target in muscles.primary_movers or muscles.secondary_movers; for higher function patient, prioritize finding those muscle target in muscles.stabiliser
   Examples: Side lying clamshell, hip abduction, side plank variations

2. Issue: Limited posterior chain flexibility (cannot touch toes)
   Strategy: Prioritize exercises with high hamstring + glute_max in muscles (value 4-5 each)
   Examples: Glute bridges, hamstring bridges, hip hinge

## Cell 5: Run LLM #1 - Exercise Recommendation Agent

LLM #1 receives the identified biomechanical targets and recommends 4 exercises based on patient capability.

In [7]:
print("\n" + "="*80)
print("Running LLM #1: Exercise Recommendation Agent")
print("="*80 + "\n")
print("Analyzing patient and selecting exercises...\n")

try:
    # Generate recommendations
    llm1_output = generate_exercise_recommendations(llm, patient_profile)
    
    # Print formatted output
    print_llm1_output(llm1_output)
    
    print("\n✓ LLM #1 recommendations complete!")
    
except Exception as e:
    print(f"❌ ERROR in LLM #1: {e}")
    raise


Running LLM #1: Exercise Recommendation Agent

Analyzing patient and selecting exercises...



Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=019c80c2-1b4c-7402-9cf5-8a72bb192951,id=019c80c2-1b4c-7402-9cf5-8a72bb192951; trace=019c80c2-1b4c-7402-9cf5-8a72bb192951,id=019c80c2-1b96-75b3-826c-851ac9771876


LLM #1: EXERCISE RECOMMENDATION AGENT

[BIOMECHANICAL TARGETS - RULE-BASED ANALYSIS]

  1. Issue: Dynamic knee instability (Valgus alignment - knock-knees)
     Strategy: Dynamic knee instability usually associates with weak core anti-rotation control, to prioritize exercises with `core_contra=true`. Prioritize exercises with high glute_med_min in muscles.primary_movers or muscles.secondary_movers (value 4-5). Match muscle role to functional capacity: for lower function patient, try to find those muscle target in muscles.primary_movers or muscles.secondary_movers; for higher function patient, prioritize finding those muscle target in muscles.stabiliser
     Examples: Side lying clamshell, hip abduction, side plank variations

  2. Issue: Limited posterior chain flexibility (cannot touch toes)
     Strategy: Prioritize exercises with high hamstring + glute_max in muscles (value 4-5 each)
     Examples: Glute bridges, hamstring bridges, hip hinge exercises

  3. Issue: Core instability (

## Cell 6: Run LLM #2 - Safety Verification Agent

LLM #2 reviews proposed exercises and verifies safety against objective clinical measures.

In [8]:
print("\n" + "="*80)
print("Running LLM #2: Safety Verification Agent")
print("="*80 + "\n")
print("Verifying safety of proposed exercises...\n")

try:
    # Verify safety and finalize
    llm2_output = verify_safety_and_finalize(llm, patient_profile, llm1_output)
    
    # Print formatted output
    print_llm2_output(llm2_output)
    
    print("\n✓ LLM #2 safety verification complete!")
    
except Exception as e:
    print(f"❌ ERROR in LLM #2: {e}")
    raise


Running LLM #2: Safety Verification Agent

Verifying safety of proposed exercises...



Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=019c80c2-1b4c-7402-9cf5-8a72bb192951,id=019c80c2-1b96-75b3-826c-851ac9771876; trace=019c80c2-1b4c-7402-9cf5-8a72bb192951,id=019c80c2-6134-7f91-b209-6d048e38c78d; trace=019c80c2-1b4c-7402-9cf5-8a72bb192951,id=019c80c2-6134-7f91-b209-6d048e38c78d; trace=019c80c2-1b4c-7402-9cf5-8a72bb192951,id=019c80c2-1b4c-7402-9cf5-8a72bb192951; trace=019c80c2-6155-7412-88ac-c5d5798ee28d,id=019c80c2-6155-7412-88ac-c5d5798ee28d; trace=019c80c2-6155-7412-88ac-c5d5798ee28d,id=019c80c2-615a-7a31-9e71-a0dc2f795116


LLM #2: SAFETY VERIFICATION AGENT

[SAFETY REVIEW]

1. Weight-Bearing Check:
   STS Performance: Below Average
   Trunk sway: present | Hip sway: present
   Risk: HIGH - high_risk
   Reasoning: Patient has Below Average STS performance (7 reps vs 11-14 benchmark) AND both trunk sway and hip sway present. According to decision logic: 'Below Average' OR (trunk_sway AND hip_sway both present): HIGH RISK → REJECT, suggest non-weight-bearing alternative.

2. Kneeling Check:
   SP5 (kneeling): 3
   Pain average: 1.33
   Risk: MODERATE - moderate_risk
   Reasoning: Kneeling score = 3 (Severe difficulty) and pain.avg = 1.33 (<3.0). According to decision logic: Kneeling score = 3: MODERATE RISK → APPROVE WITH MODIFICATIONS (thick padding, shorter holds, monitor pain).

3. Core Stability Check:
   Trunk sway: present | Hip sway: present
   F2 (standing): 4 | SP4 (twisting): 3
   Function ADL: 39.2
   Risk: HIGH - high_risk
   Reasoning: BOTH trunk and hip sway present, AND function_ADL.normalize

## Cell 7: Display Final Prescription

Final exercise prescription after safety review and modifications.

In [9]:
print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + " "*24 + "FINAL EXERCISE PRESCRIPTION" + " "*28 + "#")
print("#" + " "*78 + "#")
print("#"*80 + "\n")

print(f"Patient: {username}")
print(f"Age: {patient_profile['demographics']['age']} | Gender: {patient_profile['demographics']['gender']}")
print(f"STS Performance: {patient_profile['sts_assessment']['repetition_count']} reps "
      f"({patient_profile['sts_assessment']['benchmark_performance']} compared to benchmark ({patient_profile['sts_assessment']['age_gender_benchmark_range']}))")
print(f"Biomechanical Issues: {patient_profile['sts_assessment']['knee_alignment']} knee, "
      f"Flexibility: {patient_profile['flexibility']['toe_touch_test']} touch toes")
print("\n" + "-"*80 + "\n")

for i, ex in enumerate(llm2_output['final_prescription'], 1):
    print(f"Exercise {i}: {ex['exercise_name']}")
    print(f"           {ex['exercise_name_ch']}")
    print(f"")
    # Handle positions array (new schema v3.0)
    positions_display = ', '.join([p.replace('_', ' ').title() for p in ex['positions']])
    print(f"  Positions: {positions_display}")
    print(f"  Difficulty: {ex['difficulty']}/10")
    print(f"")
    print(f"  Clinical Rationale:")
    print(f"  {ex['clinical_rationale']}")
    
    if ex['modifications']:
        print(f"")
        print(f"  ⚠️  Safety Modifications:")
        for j, mod in enumerate(ex['modifications'], 1):
            print(f"      {j}. {mod}")
    
    print("\n" + "-"*80 + "\n")

print("\n✅ Exercise prescription complete!")
print("\nRecommendation: Review with supervising physiotherapist before patient execution.")


################################################################################
#                                                                              #
#                        FINAL EXERCISE PRESCRIPTION                            #
#                                                                              #
################################################################################

Patient: Test_05
Age: 82 | Gender: female
STS Performance: 7 reps (Below Average compared to benchmark (11 - 14))
Biomechanical Issues: valgus knee, Flexibility: cannot touch toes

--------------------------------------------------------------------------------

Exercise 1: Side lying Clamshell
           側臥蚌殼式

  Positions: Side Lying
  Difficulty: 1/10

  Clinical Rationale:
  Foundational exercise for gluteus medius/minimus strengthening to address dynamic knee valgus alignment. Core_contra=true provides contralateral core stability training. Safe side-lying position appropriate for

## Cell 8 (Optional): Export Results to JSON

In [10]:
from datetime import datetime

# Export complete results for record-keeping
output_data = {
    'patient_username': username,
    'patient_profile_summary': {
        'age': patient_profile['demographics']['age'],
        'gender': patient_profile['demographics']['gender'],
        'sts_benchmark_performance': patient_profile['sts_assessment']['benchmark_performance'],
        'knee_alignment': patient_profile['sts_assessment']['knee_alignment'],
        'flexibility': patient_profile['flexibility']['toe_touch_test']
    },
    'llm1_recommendations': llm1_output,
    'llm2_safety_review': llm2_output
}

# Save to file
output_filename = f"prescription_{username}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✓ Results exported to: {output_filename}")

✓ Results exported to: prescription_Test_05_20260221_231255.json


## Cell 9: Export User-Friendly HTML Report

Generate a comprehensive HTML report with all details for physiotherapist review.

In [11]:
from datetime import datetime

# Generate comprehensive HTML report
def safe_get(dictionary, key, default='N/A'):
    """Safely get dictionary value with default"""
    return dictionary.get(key, default)

# Calculate BMI if height and weight are available
def calculate_bmi(height_cm, weight_kg):
    """Calculate BMI from height (cm) and weight (kg)"""
    try:
        height_m = float(height_cm) / 100
        bmi = float(weight_kg) / (height_m ** 2)
        return round(bmi, 1)
    except (ValueError, ZeroDivisionError, TypeError):
        return None

# Extract demographics safely
demo = patient_profile.get('demographics', {})
age = demo.get('age', 'N/A')
gender = demo.get('gender', 'N/A')
height_cm = demo.get('height_cm', None)
weight_kg = demo.get('weight_kg', None)
bmi = calculate_bmi(height_cm, weight_kg) if height_cm and weight_kg else None

# Extract STS assessment safely
sts = patient_profile.get('sts_assessment', {})

# Extract questionnaire sections safely
questionnaire_sections = patient_profile.get('questionnaire_sections', {})

# Extract position-relevant questions safely
position_questions = patient_profile.get('position_relevant_questions', {})

# Build HTML content
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Exercise Prescription Report - {username}</title>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f5f5f5;
            color: #333;
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 30px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }}
        .header h1 {{
            margin: 0 0 10px 0;
            font-size: 32px;
        }}
        .header .timestamp {{
            font-size: 14px;
            opacity: 0.9;
        }}
        .section {{
            background: white;
            padding: 25px;
            margin-bottom: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .section-title {{
            color: #667eea;
            font-size: 24px;
            margin-top: 0;
            margin-bottom: 20px;
            padding-bottom: 10px;
            border-bottom: 3px solid #667eea;
        }}
        .subsection-title {{
            color: #764ba2;
            font-size: 18px;
            margin-top: 20px;
            margin-bottom: 10px;
            font-weight: 600;
        }}
        .info-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 15px;
            margin-bottom: 20px;
        }}
        .info-item {{
            background: #f8f9fa;
            padding: 15px;
            border-radius: 6px;
            border-left: 4px solid #667eea;
        }}
        .info-label {{
            font-weight: 600;
            color: #555;
            font-size: 14px;
            margin-bottom: 5px;
        }}
        .info-value {{
            font-size: 18px;
            color: #333;
        }}
        .exercise-card {{
            background: #f8f9fa;
            padding: 20px;
            margin-bottom: 20px;
            border-radius: 8px;
            border-left: 5px solid #667eea;
        }}
        .exercise-header {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 15px;
        }}
        .exercise-name {{
            font-size: 22px;
            font-weight: 600;
            color: #333;
        }}
        .exercise-name-ch {{
            font-size: 18px;
            color: #666;
            margin-top: 5px;
        }}
        .difficulty-badge {{
            background: #667eea;
            color: white;
            padding: 8px 15px;
            border-radius: 20px;
            font-size: 14px;
            font-weight: 600;
        }}
        .muscle-targets {{
            margin: 15px 0;
        }}
        .muscle-category {{
            display: inline-block;
            margin-right: 15px;
            margin-bottom: 10px;
            padding: 8px 12px;
            background: #e3f2fd;
            border-radius: 5px;
            font-size: 14px;
        }}
        .muscle-category.primary {{
            background: #c8e6c9;
        }}
        .muscle-category.secondary {{
            background: #fff9c4;
        }}
        .muscle-category.stabiliser {{
            background: #ffe0b2;
        }}
        .rationale {{
            background: white;
            padding: 15px;
            border-radius: 6px;
            margin: 15px 0;
            border-left: 3px solid #764ba2;
            line-height: 1.6;
        }}
        .modifications {{
            background: #fff3cd;
            padding: 15px;
            border-radius: 6px;
            border-left: 4px solid #ffc107;
        }}
        .modifications-title {{
            font-weight: 600;
            color: #856404;
            margin-bottom: 10px;
            font-size: 16px;
        }}
        .modifications ul {{
            margin: 0;
            padding-left: 20px;
        }}
        .modifications li {{
            margin-bottom: 8px;
            color: #856404;
        }}
        .safety-review {{
            background: #e8f5e9;
            padding: 15px;
            border-radius: 6px;
            margin-bottom: 15px;
        }}
        .risk-badge {{
            display: inline-block;
            padding: 5px 12px;
            border-radius: 15px;
            font-size: 13px;
            font-weight: 600;
            margin-left: 10px;
        }}
        .risk-low {{
            background: #4caf50;
            color: white;
        }}
        .risk-moderate {{
            background: #ff9800;
            color: white;
        }}
        .risk-high {{
            background: #f44336;
            color: white;
        }}
        .decision-badge {{
            display: inline-block;
            padding: 8px 15px;
            border-radius: 20px;
            font-weight: 600;
            font-size: 14px;
            margin-bottom: 10px;
        }}
        .decision-approved {{
            background: #4caf50;
            color: white;
        }}
        .decision-modified {{
            background: #ff9800;
            color: white;
        }}
        .decision-rejected {{
            background: #f44336;
            color: white;
        }}
        .biomechanical-target {{
            background: white;
            padding: 15px;
            margin-bottom: 15px;
            border-radius: 6px;
            border-left: 4px solid #667eea;
        }}
        .target-issue {{
            font-weight: 600;
            color: #667eea;
            margin-bottom: 8px;
        }}
        .target-strategy {{
            color: #555;
            margin-bottom: 8px;
            line-height: 1.5;
        }}
        .target-examples {{
            color: #666;
            font-style: italic;
            font-size: 14px;
        }}
        .rule-based-badge {{
            display: inline-block;
            background: #4caf50;
            color: white;
            padding: 5px 10px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: 600;
            margin-left: 10px;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 15px 0;
        }}
        th, td {{
            padding: 12px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        th {{
            background: #f8f9fa;
            font-weight: 600;
            color: #555;
        }}
        .print-button {{
            background: #667eea;
            color: white;
            padding: 12px 24px;
            border: none;
            border-radius: 6px;
            font-size: 16px;
            cursor: pointer;
            margin-bottom: 20px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.2);
        }}
        .print-button:hover {{
            background: #5568d3;
        }}
        @media print {{
            body {{
                background: white;
            }}
            .print-button {{
                display: none;
            }}
            .section {{
                box-shadow: none;
                page-break-inside: avoid;
            }}
        }}
    </style>
</head>
<body>
    <button class="print-button" onclick="window.print()">🖨️ Print Report</button>
    
    <div class="header">
        <h1>🏥 Exercise Prescription Report</h1>
        <div class="timestamp">Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</div>
    </div>

    <!-- Patient Overview -->
    <div class="section">
        <h2 class="section-title">📋 Patient Overview</h2>
        <div class="info-grid">
            <div class="info-item">
                <div class="info-label">Patient ID</div>
                <div class="info-value">{username}</div>
            </div>
            <div class="info-item">
                <div class="info-label">Age / Gender</div>
                <div class="info-value">{age} years / {str(gender).capitalize()}</div>
            </div>"""

if height_cm and weight_kg:
    html_content += f"""
            <div class="info-item">
                <div class="info-label">Height / Weight</div>
                <div class="info-value">{height_cm} cm / {weight_kg} kg</div>
            </div>"""

if bmi:
    html_content += f"""
            <div class="info-item">
                <div class="info-label">BMI</div>
                <div class="info-value">{bmi}</div>
            </div>"""

html_content += """
        </div>
    </div>

    <!-- STS Assessment -->
    <div class="section">
        <h2 class="section-title">💪 Sit-to-Stand (STS) Assessment</h2>
        <div class="info-grid">"""

html_content += f"""
            <div class="info-item">
                <div class="info-label">Repetitions</div>
                <div class="info-value">{sts.get('repetition_count', 'N/A')} reps</div>
            </div>
            <div class="info-item">
                <div class="info-label">Performance Level</div>
                <div class="info-value">{sts.get('benchmark_performance', 'N/A')}</div>
            </div>
            <div class="info-item">
                <div class="info-label">HK Benchmark Range</div>
                <div class="info-value">{sts.get('age_gender_benchmark_range', 'N/A')}</div>
            </div>
            <div class="info-item">
                <div class="info-label">Knee Alignment</div>
                <div class="info-value">{str(sts.get('knee_alignment', 'N/A')).capitalize()}</div>
            </div>
            <div class="info-item">
                <div class="info-label">Trunk Sway</div>
                <div class="info-value">{str(sts.get('trunk_sway', 'N/A')).capitalize()}</div>
            </div>
            <div class="info-item">
                <div class="info-label">Hip Sway</div>
                <div class="info-value">{str(sts.get('hip_sway', 'N/A')).capitalize()}</div>
            </div>
        </div>
    </div>

    <!-- Questionnaire Results -->
    <div class="section">
        <h2 class="section-title">📝 KOOS Questionnaire Results</h2>
        <p style="color: #666; margin-bottom: 20px;">All scores normalized to 0-100 scale (higher = better function)</p>
        <table>
            <thead>
                <tr>
                    <th>Category</th>
                    <th>Score (0-100)</th>
                    <th>Average Item Score</th>
                </tr>
            </thead>
            <tbody>"""

# Add questionnaire scores
for category, data in questionnaire_sections.items():
    html_content += f"""
                <tr>
                    <td>{category.replace('_', ' ').title()}</td>
                    <td><strong>{data.get('normalized_0_100', 0):.1f}</strong></td>
                    <td>{data.get('avg', 0):.2f}</td>
                </tr>"""

html_content += """
            </tbody>
        </table>
        
        <h3 class="subsection-title">Position-Relevant Questions</h3>
        <p style="color: #666; font-size: 14px;">Scale: 0=None, 1=Mild, 2=Moderate, 3=Severe, 4=Extreme</p>
        <table>
            <thead>
                <tr>
                    <th>Category</th>
                    <th>Question</th>
                    <th>Score</th>
                </tr>
            </thead>
            <tbody>"""

# Weight-bearing questions
wb_spectrum = position_questions.get('weight_bearing_spectrum', {})
for q in wb_spectrum.get('questions', []):
    html_content += f"""
                <tr>
                    <td>Weight-bearing</td>
                    <td>{q.get('question', 'N/A')}</td>
                    <td><strong>{q.get('score', 'N/A')}</strong></td>
                </tr>"""

# Kneeling questions
quad_questions = position_questions.get('quadruped', {})
for q in quad_questions.get('questions', []):
    html_content += f"""
                <tr>
                    <td>Quadruped</td>
                    <td>{q.get('question', 'N/A')}</td>
                    <td><strong>{q.get('score', 'N/A')}</strong></td>
                </tr>"""

flexibility = patient_profile.get('flexibility', {})
html_content += f"""
            </tbody>
        </table>
        
        <div class="info-grid" style="margin-top: 20px;">
            <div class="info-item">
                <div class="info-label">Flexibility (Toe Touch)</div>
                <div class="info-value">{str(flexibility.get('toe_touch_test', 'N/A')).capitalize()}</div>
            </div>
        </div>
    </div>

    <!-- Biomechanical Targets (Rule-Based) -->
    <div class="section">
        <h2 class="section-title">🎯 Biomechanical Targets<span class="rule-based-badge">RULE-BASED</span></h2>
        <p style="color: #666; margin-bottom: 20px;">These targets were identified using rule-based clinical criteria before LLM analysis</p>
"""

# Use the new structure: llm1_output['biomechanical_targets'] instead of llm1_output['patient_assessment']['biomechanical_targets']
biomechanical_targets = llm1_output.get('biomechanical_targets', [])
if biomechanical_targets:
    for target in biomechanical_targets:
        html_content += f"""
        <div class="biomechanical-target">
            <div class="target-issue">🎯 {target['issue']}</div>
            <div class="target-strategy"><strong>Strategy:</strong> {target['strategy']}</div>
            <div class="target-examples"><strong>Examples:</strong> {', '.join(target['examples'])}</div>
        </div>"""
else:
    html_content += """
        <p style="color: #666;">No specific biomechanical targets identified for this patient.</p>
"""

html_content += """
    </div>

    <!-- LLM 1: Patient Assessment -->
    <div class="section">
        <h2 class="section-title">🤖 LLM #1: Clinical Assessment & Exercise Selection</h2>
        
        <h3 class="subsection-title">Capability Summary</h3>
        <div class="rationale">
            {llm1_output['patient_assessment']['capability_summary']}
        </div>
"""

html_content += f"""
        <div class="info-grid" style="margin-top: 20px;">
            <div class="info-item">
                <div class="info-label">Recommended Positions</div>
                <div class="info-value">{', '.join([p.replace('_', ' ').title() for p in llm1_output['patient_assessment']['recommended_positions']])}</div>
            </div>
            <div class="info-item">
                <div class="info-label">Difficulty Range</div>
                <div class="info-value">{llm1_output['patient_assessment']['difficulty_range']}</div>
            </div>
        </div>
        
        <h3 class="subsection-title">Selected Exercises (Pre-Safety Review)</h3>"""

# LLM1 Selected Exercises
for i, ex in enumerate(llm1_output['selected_exercises'], 1):
    primary_muscles = ', '.join(ex['muscle_targets']['primary']) if ex['muscle_targets']['primary'] else 'None'
    secondary_muscles = ', '.join(ex['muscle_targets']['secondary']) if ex['muscle_targets']['secondary'] else 'None'
    stabiliser_muscles = ', '.join(ex['muscle_targets']['stabiliser']) if ex['muscle_targets']['stabiliser'] else 'None'
    
    html_content += f"""
        <div class="exercise-card">
            <div class="exercise-header">
                <div>
                    <div class="exercise-name">{i}. {ex['exercise_name']}</div>
                    <div class="exercise-name-ch">{ex['exercise_name_ch']}</div>
                </div>
                <div class="difficulty-badge">Difficulty: {ex['difficulty']}/10</div>
            </div>
            <div><strong>Exercise ID:</strong> {ex['exercise_id']}</div>
            <div><strong>Positions:</strong> {', '.join([p.replace('_', ' ').title() for p in ex['positions']])}</div>
            <div class="muscle-targets">
                <div class="muscle-category primary"><strong>Primary:</strong> {primary_muscles}</div>
                <div class="muscle-category secondary"><strong>Secondary:</strong> {secondary_muscles}</div>
                <div class="muscle-category stabiliser"><strong>Stabiliser:</strong> {stabiliser_muscles}</div>
            </div>
            <div class="rationale">
                <strong>Reasoning:</strong> {ex['reasoning']}
            </div>
        </div>"""

html_content += """
    </div>

    <!-- LLM 2: Safety Review -->
    <div class="section">
        <h2 class="section-title">🛡️ LLM #2: Safety Verification</h2>
        
        <h3 class="subsection-title">Safety Checks</h3>"""

# Weight-bearing check
wb_check = llm2_output['safety_review']['weight_bearing_check']
wb_risk_class = wb_check['risk_level'].lower()
html_content += f"""
        <div class="safety-review">
            <strong>1. Weight-Bearing Check</strong>
            <span class="risk-badge risk-{wb_risk_class}">{wb_check['risk_level'].upper()} RISK</span>
            <div style="margin-top: 10px;">
                <strong>Objective Data:</strong><br>
                • STS Performance: {wb_check['objective_data']['sts_benchmark_performance']}<br>
                • Trunk Sway: {wb_check['objective_data']['trunk_sway']}<br>
                • Hip Sway: {wb_check['objective_data']['hip_sway']}
            </div>
            <div style="margin-top: 10px;">
                <strong>Reasoning:</strong> {wb_check['reasoning']}
            </div>
        </div>"""

# Kneeling check
kn_check = llm2_output['safety_review']['kneeling_check']
kn_risk_class = kn_check['risk_level'].lower()
html_content += f"""
        <div class="safety-review">
            <strong>2. Kneeling Check</strong>
            <span class="risk-badge risk-{kn_risk_class}">{kn_check['risk_level'].upper()} RISK</span>
            <div style="margin-top: 10px;">
                <strong>Objective Data:</strong><br>
                • SP5 (Kneeling): {kn_check['objective_data']['sp5_kneeling']}<br>
                • Pain Average: {kn_check['objective_data']['pain_avg']:.2f}
            </div>
            <div style="margin-top: 10px;">
                <strong>Reasoning:</strong> {kn_check['reasoning']}
            </div>
        </div>"""

# Core stability check
cs_check = llm2_output['safety_review']['core_stability_check']
cs_risk_class = cs_check['risk_level'].lower()
html_content += f"""
        <div class="safety-review">
            <strong>3. Core Stability Check</strong>
            <span class="risk-badge risk-{cs_risk_class}">{cs_check['risk_level'].upper()} RISK</span>
            <div style="margin-top: 10px;">
                <strong>Objective Data:</strong><br>
                • Trunk Sway: {cs_check['objective_data']['trunk_sway']}<br>
                • Hip Sway: {cs_check['objective_data']['hip_sway']}<br>
                • F2 (Standing): {cs_check['objective_data']['f2_standing']}<br>
                • SP4 (Twisting): {cs_check['objective_data']['sp4_twisting']}<br>
                • Function ADL Score: {cs_check['objective_data']['function_ADL_normalized']:.1f}
            </div>
            <div style="margin-top: 10px;">
                <strong>Reasoning:</strong> {cs_check['reasoning']}
            </div>
        </div>"""

html_content += """
        <h3 class="subsection-title">Exercise-by-Exercise Decisions</h3>"""

# Exercise decisions
for decision in llm2_output['exercise_decisions']:
    decision_class = 'approved' if decision['decision'] == 'APPROVED' else 'modified' if 'MODIFICATIONS' in decision['decision'] else 'rejected'
    html_content += f"""
        <div class="exercise-card">
            <div class="exercise-header">
                <div class="exercise-name">{decision['exercise_name']} (ID: {decision['exercise_id']})</div>
                <div class="decision-badge decision-{decision_class}">{decision['decision']}</div>
            </div>"""
    
    if decision['safety_constraints_triggered']:
        html_content += f"""
            <div style="margin: 10px 0;">
                <strong>⚠️ Constraints Triggered:</strong> {', '.join(decision['safety_constraints_triggered'])}
            </div>"""
    
    html_content += f"""
            <div class="rationale">
                <strong>Reasoning:</strong> {decision['reasoning']}
            </div>"""
    
    if decision['modifications']:
        html_content += """
            <div class="modifications">
                <div class="modifications-title">Required Modifications:</div>
                <ul>"""
        for mod in decision['modifications']:
            html_content += f"<li>{mod}</li>"
        html_content += """
                </ul>
            </div>"""
    
    if decision.get('replacement_suggestion'):
        html_content += f"""
            <div style="margin-top: 10px; padding: 10px; background: #ffe0e0; border-radius: 6px;">
                <strong>Replacement Suggestion:</strong> {decision['replacement_suggestion']}
            </div>"""
    
    html_content += """
        </div>"""

html_content += """
    </div>

    <!-- Final Prescription -->
    <div class="section">
        <h2 class="section-title">✅ Final Exercise Prescription</h2>
        <p style="background: #fff3cd; padding: 15px; border-radius: 6px; border-left: 4px solid #ffc107;">
            <strong>⚠️ Important:</strong> This prescription should be reviewed by a supervising physiotherapist before patient execution.
        </p>"""

# Final prescription exercises
for i, ex in enumerate(llm2_output['final_prescription'], 1):
    html_content += f"""
        <div class="exercise-card">
            <div class="exercise-header">
                <div>
                    <div class="exercise-name">{i}. {ex['exercise_name']}</div>
                    <div class="exercise-name-ch">{ex['exercise_name_ch']}</div>
                </div>
                <div class="difficulty-badge">Difficulty: {ex['difficulty']}/10</div>
            </div>
            <div><strong>Exercise ID:</strong> {ex['exercise_id']}</div>
            <div><strong>Positions:</strong> {', '.join([p.replace('_', ' ').title() for p in ex['positions']])}</div>
            <div class="rationale">
                <strong>Clinical Rationale:</strong><br>
                {ex['clinical_rationale']}
            </div>"""
    
    if ex['modifications']:
        html_content += """
            <div class="modifications">
                <div class="modifications-title">⚠️ Safety Modifications:</div>
                <ul>"""
        for mod in ex['modifications']:
            html_content += f"<li>{mod}</li>"
        html_content += """
                </ul>
            </div>"""
    
    html_content += """
        </div>"""

html_content += """
    </div>

    <div style="text-align: center; padding: 20px; color: #666; font-size: 14px;">
        <p>Report generated by PhysioAIign LLM-Based Exercise Recommendation System</p>
        <p>Two-LLM Architecture: Rule-Based Biomechanical Analysis + LLM Exercise Selection + LLM Safety Verification</p>
    </div>

</body>
</html>
"""

# Save HTML file
html_filename = f"prescription_report_{username}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
with open(html_filename, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✓ User-friendly HTML report exported to: {html_filename}")
print(f"  Open this file in a web browser for easy reading and printing")
print(f"  All JSON data is included in a formatted, readable layout")

✓ User-friendly HTML report exported to: prescription_report_Test_05_20260221_231255.html
  Open this file in a web browser for easy reading and printing
  All JSON data is included in a formatted, readable layout


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=019c80c2-6155-7412-88ac-c5d5798ee28d,id=019c80c2-615a-7a31-9e71-a0dc2f795116; trace=019c80c2-6155-7412-88ac-c5d5798ee28d,id=019c80c2-cbae-7211-89c3-f1c240c5f897; trace=019c80c2-6155-7412-88ac-c5d5798ee28d,id=019c80c2-cbae-7211-89c3-f1c240c5f897; trace=019c80c2-6155-7412-88ac-c5d5798ee28d,id=019c80c2-6155-7412-88ac-c5d5798ee28d
